In [8]:
import random
from openai import OpenAI

from tqdm import tqdm


from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders.text import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter

from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

In [9]:
loader = DirectoryLoader("../data/from_site/md/faq", glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding':'utf-8'})
docs = loader.load()
len(docs)

14

In [10]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

In [11]:
docs[0].dict()

{'id': None,
 'metadata': {'source': '..\\data\\from_site\\md\\faq\\FAQ_ RUTUBE для Smart TV.md'},
 'page_content': '## Что такое Smart TV?\n\nЭто телевизор или ТВ-приставка с доступом к интернету и возможностью скачивать\nсторонние приложения и сервисы. Приложение RUTUBE можно установить почти на\nлюбое устройство Smart TV.\n\n## Для каких телевизоров и ТВ-приставок доступно приложение RUTUBE?\n\nПриложение RUTUBE доступно для телевизоров Samsung (с 2015 г. выпуска), LG (c\n2014 г. выпуска), Sony, Xiaomi, Haier, BBK и других телевизоров и ТВ-приставок\nпод управлением Android TV, телевизоров Hisense и других ТВ под управлением ОС\nVidaa; Яндекс ТВ, Sber Devices, Philips, Apple TV. [Подробнее\n](https://rutube.ru/smarttv/)\n\n## Для каких телевизоров и ТВ-приставок доступно приложение RUTUBE Детям?\n\nRUTUBE Детям пока недоступен в виде отдельного приложения. Сейчас детский\nконтент расположен разделе «Детям» в основном приложении RUTUBE.\n\n## Где можно скачать приложение RUTUBE для S

In [12]:
chunks = []

for doc in docs:
    metadata = doc.metadata
    page_content = doc.page_content
    doc_chunks = markdown_splitter.split_text(page_content)
    for chunk in doc_chunks:
        chunk.metadata = chunk.metadata | metadata
        chunks.append(chunk)

In [13]:
len(chunks)

270

In [14]:
def create_chunk_pair(chunk_0:str, chunk_1:str) -> str:
    result = f'''КУСОК 1:\n=====\n{chunk_0}\n=====\n\nКУСОК 2\n=====\n{chunk_1}\n=====\n'''.strip()
    return result


# SYSTEM_PROMPT = "Тебе прислали 2 куска текста, придумай вопрос, ответ на который требовал бы информации из двух чанков сразу. Вопросы должны относиться с сервису RUTUBE. И быть написаны, будто бы они идут от клиента специалисту службы поддержки."
SYSTEM_PROMPT = "Тебе прислали 2 куска текста, придумай вопрос, ответ на который требовал бы информации из двух чанков сразу. Вопросы должны относиться с сервису RUTUBE. И быть написаны, будто бы они идут от клиента специалисту службы поддержки. Вопрос должен быть один, чтобы ответ на него сочетал информацию из разных кусков."


client = OpenAI()
def complete(chunk_pair: str) :
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": chunk_pair
            }
        ]
    )
    return completion

def extract_content(completion) -> str:
    try:
        return completion.choices[0].message.content
    except:
        return '==ERROR=='

In [15]:
def create_chunk_from_document(doc) -> str:
    headers = '\n'.join([doc.metadata[meta] for meta in doc.metadata if meta.startswith('Header')])
    page_content = doc.page_content
    result = headers + '\n' + page_content
    return result

In [16]:
def generate_answers(documents_list:list[tuple,...]) -> list[dict]:
    result = []
    for doc_a, doc_b in tqdm(documents_list):
        sub_result = {}
        chunk_a = create_chunk_from_document(doc_a)
        chunk_b = create_chunk_from_document(doc_b)
        chunk_pair = create_chunk_pair(chunk_a,chunk_b)
        question_completion = complete(chunk_pair)
        question = extract_content(question_completion)
        sub_result['question'] = question
        sub_result['chunk_a'] = chunk_a
        sub_result['chunk_b'] = chunk_b
        sub_result['metadata'] = {'answer': question_completion.to_dict()} | {'doc_a':doc_a.dict()} | {'doc_b': doc_b.dict()}
        result.append(sub_result)

    return result
        


In [17]:
def get_random_pairs(lst, num_pairs):
    all_pairs = [(a, b) for i, a in enumerate(lst) for b in lst[i+1:]]
    
    if num_pairs > len(all_pairs):
        raise ValueError("Количество пар больше, чем возможные уникальные комбинации.")
    
    return random.sample(all_pairs, num_pairs)

# my_list = [x for x in range(1000)]
# num_pairs = 50
# random_pairs = get_random_pairs(my_list, num_pairs)
# print(random_pairs)


In [18]:
chunks[0]

Document(metadata={'Header 2': 'Что такое Smart TV?', 'source': '..\\data\\from_site\\md\\faq\\FAQ_ RUTUBE для Smart TV.md'}, page_content='Это телевизор или ТВ-приставка с доступом к интернету и возможностью скачивать\nсторонние приложения и сервисы. Приложение RUTUBE можно установить почти на\nлюбое устройство Smart TV.')

In [19]:
result = generate_answers(get_random_pairs(chunks, 100))

100%|██████████| 100/100 [02:22<00:00,  1.43s/it]


In [20]:
result

[{'question': 'Почему моя заявка на монетизацию была отклонена, если я разместил видео с YouTube в максимальном качестве 1080р, но не учел правила платформы?',
  'chunk_a': 'Монетизация на RUTUBE\nПочему заявка может быть отклонена?\nДля подключения монетизации размещенный на канале контент должен\nсоответствовать правилам платформы в части п.4.6, п.4.7 и п.4.8 :\n<https://rutube.ru/info/agreement>',
  'chunk_b': 'В каком качестве видео переносятся с YouTube на RUTUBE?\nВидео переносятся в том же качестве, в котором опубликованы на YouTube\n(максимальное возможное возможное качество - 1080р).',
  'metadata': {'answer': {'id': 'chatcmpl-ABfUgJ8zmd1Te6mCzX6EhsvAwXEiO',
    'choices': [{'finish_reason': 'stop',
      'index': 0,
      'logprobs': None,
      'message': {'content': 'Почему моя заявка на монетизацию была отклонена, если я разместил видео с YouTube в максимальном качестве 1080р, но не учел правила платформы?',
       'refusal': None,
       'role': 'assistant'}}],
    'creat

In [21]:
import json

with open('synth_complex_questions_2.json', 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

print("Данные успешно сохранены в файл data.json")


Данные успешно сохранены в файл data.json
